In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.insert(0, '/content/drive/MyDrive/pd-interpretability')

In [ ]:
# install dependencies
!pip install -q transformers datasets librosa scipy tqdm

In [ ]:
import torch
import numpy as np
import json
from pathlib import Path
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

# verify gpu
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## 1. Load Fine-tuned Model and Dataset

In [ ]:
# configuration
CONFIG = {
    'model_path': '/content/drive/MyDrive/pd-interpretability/results/checkpoints/best_model',
    'data_path': '/content/drive/MyDrive/pd-interpretability/data',
    'output_path': '/content/drive/MyDrive/pd-interpretability/results/patching',
    'n_pairs': 100,  # number of (HC, PD) pairs for patching
    'batch_size': 8,
    'random_seed': 42
}

# create output directory
output_path = Path(CONFIG['output_path'])
output_path.mkdir(parents=True, exist_ok=True)

np.random.seed(CONFIG['random_seed'])
torch.manual_seed(CONFIG['random_seed'])

In [ ]:
from src.models import Wav2Vec2PDClassifier
from src.data import ItalianPVSDataset

# load fine-tuned model
model_path = Path(CONFIG['model_path'])

if model_path.exists():
    classifier = Wav2Vec2PDClassifier.load(model_path)
    print(f"loaded model from {model_path}")
else:
    # load base model for testing
    print("fine-tuned model not found, loading base model")
    classifier = Wav2Vec2PDClassifier(num_labels=2)

# load dataset
dataset = ItalianPVSDataset(
    root_dir=Path(CONFIG['data_path']) / 'raw' / 'italian_pvs',
    max_duration=10.0
)

print(f"dataset size: {len(dataset)}")
print(f"model layers: {len(classifier.model.wav2vec2.encoder.layers)}")

## 2. Create Minimal Pairs for Patching

Match HC samples with acoustically similar PD samples using MFCC distance.

In [ ]:
from src.interpretability import create_mfcc_matched_pairs, create_minimal_pairs

# create mfcc-matched pairs
print("creating mfcc-matched minimal pairs...")
pairs = create_mfcc_matched_pairs(
    dataset,
    n_pairs=CONFIG['n_pairs'],
    same_task=True
)

if len(pairs) < CONFIG['n_pairs'] // 2:
    print("falling back to random matching")
    pairs = create_minimal_pairs(dataset, n_pairs=CONFIG['n_pairs'])

print(f"created {len(pairs)} minimal pairs")

# separate components
clean_inputs = [p[0] for p in pairs]
corrupted_inputs = [p[1] for p in pairs]
labels = [p[2] for p in pairs]

# show distance distribution if available
if len(pairs[0]) > 3:
    distances = [p[3] for p in pairs]
    plt.figure(figsize=(8, 4))
    plt.hist(distances, bins=30, edgecolor='black')
    plt.xlabel('MFCC Distance')
    plt.ylabel('Count')
    plt.title('Distribution of MFCC Distances in Minimal Pairs')
    plt.savefig(output_path / 'mfcc_distances.png', dpi=150, bbox_inches='tight')
    plt.show()

## 3. Layer-Level Patching

For each layer (1-12):
- Run model on HC sample, cache layer activations
- Run model on matched PD sample with patched HC activations
- Measure: Does prediction shift toward HC?

In [ ]:
from src.interpretability import ActivationPatcher

# initialize patcher
patcher = ActivationPatcher(classifier.model, device=device)

print(f"number of layers: {patcher.num_layers}")
print(f"number of attention heads: {patcher.num_heads}")
print(f"hidden size: {patcher.hidden_size}")

In [ ]:
# run layer-level patching
print("running layer-level patching...")

patching_results = patcher.run_batch_patching(
    clean_inputs,
    corrupted_inputs,
    labels,
    include_heads=False  # layer-level only first
)

layer_patching = patching_results['layer_patching']

# display results
print("\nlayer-level patching results:")
print("-" * 50)
for layer_idx in range(patcher.num_layers):
    stats = layer_patching[layer_idx]
    print(f"layer {layer_idx:2d}: mean recovery = {stats['mean_recovery']:.3f} ± {stats['std_recovery']:.3f}")

In [ ]:
# visualize layer-level patching results
layers = list(range(patcher.num_layers))
mean_recoveries = [layer_patching[l]['mean_recovery'] for l in layers]
std_recoveries = [layer_patching[l]['std_recovery'] for l in layers]

fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.bar(layers, mean_recoveries, yerr=std_recoveries, capsize=3,
              color='steelblue', edgecolor='black', alpha=0.8)

# highlight important layers (recovery > 0.1)
for i, (layer, recovery) in enumerate(zip(layers, mean_recoveries)):
    if recovery > 0.1:
        bars[i].set_color('darkred')

ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.axhline(y=0.1, color='red', linestyle='--', linewidth=1, alpha=0.7, label='importance threshold')

ax.set_xlabel('Layer', fontsize=12)
ax.set_ylabel('Logit Difference Recovery', fontsize=12)
ax.set_title('Layer-Level Activation Patching Results\n(HC → PD)', fontsize=14)
ax.set_xticks(layers)
ax.legend()

plt.tight_layout()
plt.savefig(output_path / 'layer_patching_results.png', dpi=150, bbox_inches='tight')
plt.show()

# identify important layers
important_layers = [l for l, r in zip(layers, mean_recoveries) if r > 0.1]
print(f"\nimportant layers (recovery > 0.1): {important_layers}")

## 4. Attention Head-Level Patching

For layers identified as important, patch each attention head individually.

In [ ]:
# run head-level patching on important layers
print(f"running head-level patching on layers: {important_layers}")

head_results = []

for clean, corrupted, label in zip(clean_inputs[:50], corrupted_inputs[:50], labels[:50]):
    try:
        result = patcher.run_head_patching(
            clean, corrupted, label,
            target_layers=important_layers if important_layers else None
        )
        head_results.append(result)
    except Exception as e:
        print(f"head patching failed: {e}")

print(f"completed {len(head_results)} head patching experiments")

In [ ]:
from src.interpretability import HeadImportanceRanking

# create head importance ranking
head_ranking = HeadImportanceRanking.from_patching_results(
    head_results,
    top_k=20,
    threshold=0.05
)

print("\ntop 20 important attention heads:")
print("-" * 50)
for i, (layer, head, score) in enumerate(head_ranking.head_rankings[:20]):
    print(f"{i+1:2d}. Layer {layer:2d}, Head {head:2d}: recovery = {score:.4f}")

print(f"\nidentified {len(head_ranking.important_heads)} important heads")

In [ ]:
# visualize head importance as heatmap
n_layers = patcher.num_layers
n_heads = patcher.num_heads

head_matrix = np.zeros((n_layers, n_heads))

for (layer, head), score in head_ranking.head_scores.items():
    head_matrix[layer, head] = score

fig, ax = plt.subplots(figsize=(14, 8))

im = ax.imshow(head_matrix, cmap='RdBu_r', aspect='auto', vmin=-0.2, vmax=0.2)

ax.set_xlabel('Attention Head', fontsize=12)
ax.set_ylabel('Layer', fontsize=12)
ax.set_title('Attention Head Patching Results\n(Logit Difference Recovery)', fontsize=14)

ax.set_xticks(range(n_heads))
ax.set_yticks(range(n_layers))

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Recovery Score', fontsize=11)

# mark important heads
for layer, head in head_ranking.important_heads:
    ax.add_patch(plt.Rectangle((head-0.5, layer-0.5), 1, 1, 
                               fill=False, edgecolor='black', linewidth=2))

plt.tight_layout()
plt.savefig(output_path / 'head_patching_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Mean Ablation Validation

Complement patching with ablation: replace target component activations with dataset mean.

In [ ]:
# compute mean activations across dataset
print("computing mean activations for ablation...")
mean_acts = patcher.compute_mean_activations(dataset, max_samples=200)
print(f"computed mean activations for {len(mean_acts)} layers")

In [ ]:
# run ablation validation
print("\nvalidating with mean ablation...")

ablation_results = patcher.validate_with_ablation(
    dataset,
    patching_results,
    max_samples=100
)

print("\nablation effects per layer:")
print("-" * 50)
for layer_idx, stats in ablation_results['ablation_effects'].items():
    print(f"layer {layer_idx:2d}: mean effect = {stats['mean_effect']:.4f} ± {stats['std_effect']:.4f}")

print("\nconcordance analysis:")
print(f"  spearman correlation: {ablation_results['concordance']['spearman_correlation']:.3f}")
print(f"  p-value: {ablation_results['concordance']['p_value']:.4f}")
print(f"  interpretation: {ablation_results['concordance']['interpretation']}")

In [ ]:
# visualize patching vs ablation concordance
patching_scores = [layer_patching[l]['mean_recovery'] for l in range(n_layers)]
ablation_scores = [ablation_results['ablation_effects'].get(l, {}).get('mean_effect', 0) 
                   for l in range(n_layers)]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# comparison bar plot
x = np.arange(n_layers)
width = 0.35

ax1.bar(x - width/2, patching_scores, width, label='Patching Recovery', color='steelblue')
ax1.bar(x + width/2, ablation_scores, width, label='Ablation Effect', color='coral')

ax1.set_xlabel('Layer')
ax1.set_ylabel('Effect Size')
ax1.set_title('Patching vs. Ablation Effects')
ax1.set_xticks(x)
ax1.legend()

# scatter plot for correlation
ax2.scatter(patching_scores, ablation_scores, s=100, c='darkgreen', alpha=0.7)

for i, (p, a) in enumerate(zip(patching_scores, ablation_scores)):
    ax2.annotate(f'L{i}', (p, a), xytext=(5, 5), textcoords='offset points')

# add trend line
z = np.polyfit(patching_scores, ablation_scores, 1)
p = np.poly1d(z)
x_line = np.linspace(min(patching_scores), max(patching_scores), 100)
ax2.plot(x_line, p(x_line), 'r--', alpha=0.7, label='trend')

ax2.set_xlabel('Patching Recovery')
ax2.set_ylabel('Ablation Effect')
ax2.set_title(f'Concordance (r = {ablation_results["concordance"]["spearman_correlation"]:.3f})')
ax2.legend()

plt.tight_layout()
plt.savefig(output_path / 'patching_ablation_concordance.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Clinical Feature Path Patching

Test if heads with high clinical feature probing accuracy are the same heads that causally affect predictions.

In [ ]:
# load clinical features if available
clinical_path = Path(CONFIG['data_path']) / 'clinical_features' / 'italian_pvs_features.json'

if clinical_path.exists():
    with open(clinical_path, 'r') as f:
        clinical_data = json.load(f)
    
    clinical_features = {
        name: np.array(values) for name, values in clinical_data['features'].items()
    }
    sample_ids = clinical_data['sample_ids']
    
    print(f"loaded clinical features: {list(clinical_features.keys())}")
else:
    print("clinical features not found, skipping stratified analysis")
    clinical_features = None
    sample_ids = None

In [ ]:
if clinical_features is not None:
    from src.interpretability import ClinicalStratifiedPatcher
    
    stratified_patcher = ClinicalStratifiedPatcher(
        patcher,
        clinical_features,
        sample_ids
    )
    
    # test on jitter (a key PD biomarker)
    if 'jitter' in clinical_features:
        print("running stratified head patching for jitter...")
        
        jitter_results = stratified_patcher.run_stratified_head_patching(
            dataset,
            feature_name='jitter',
            target_heads=head_ranking.important_heads[:10],  # top 10 heads
            n_pairs_per_stratum=15
        )
        
        print("\njitter-stratified patching results:")
        print("-" * 50)
        for stratum, data in jitter_results.items():
            if stratum in ['low', 'medium', 'high']:
                print(f"\n{stratum.upper()} jitter stratum:")
                for head, stats in data.get('head_effects', {}).items():
                    print(f"  {head}: recovery = {stats['mean_recovery']:.4f}")
        
        # differential effects
        print("\ndifferential effects (high - low):")
        for head, diff in jitter_results.get('differential_effects', {}).items():
            print(f"  {head}: {diff:+.4f}")

## 7. Save Results

In [ ]:
# compile all results
full_results = {
    'config': CONFIG,
    'layer_patching': patching_results['layer_patching'],
    'head_patching': head_ranking.to_dict(),
    'ablation_validation': ablation_results,
    'important_layers': important_layers,
    'important_heads': [{'layer': l, 'head': h} for l, h in head_ranking.important_heads]
}

if clinical_features is not None and 'jitter' in clinical_features:
    full_results['clinical_stratified'] = {'jitter': jitter_results}

# save to json
results_path = output_path / 'patching_results.json'
with open(results_path, 'w') as f:
    json.dump(full_results, f, indent=2, default=str)

print(f"results saved to {results_path}")

In [ ]:
# summary
print("\n" + "=" * 60)
print("ACTIVATION PATCHING ANALYSIS SUMMARY")
print("=" * 60)
print(f"\ntotal minimal pairs tested: {len(pairs)}")
print(f"\nimportant layers (recovery > 0.1): {important_layers}")
print(f"number of important attention heads: {len(head_ranking.important_heads)}")

print(f"\ntop 5 attention heads:")
for layer, head, score in head_ranking.head_rankings[:5]:
    print(f"  Layer {layer}, Head {head}: {score:.4f}")

print(f"\npatching-ablation concordance: {ablation_results['concordance']['spearman_correlation']:.3f}")
print(f"concordance interpretation: {ablation_results['concordance']['interpretation']}")
print("\n" + "=" * 60)